In [2]:
from vowpalwabbit import pyvw

In [3]:
model = pyvw.vw(quiet=True)

In [4]:
model

In [5]:
train_examples = [
  "0 | price:.23 sqft:.25 age:.05 2006",
  "1 | price:.18 sqft:.15 age:.35 1976",
  "0 | price:.53 sqft:.32 age:.87 1924",
]

for example in train_examples:
    model.learn(example)


In [10]:
test_example = "| price:.46 sqft:.4 age:.10 1924"

prediction = model.predict(test_example)
print(prediction)


0.0


In [ ]:
model.predict()

In [43]:
import math
import types
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from tqdm import tqdm


def df_to_vw_regression(df, filepath='in.txt', sample_weights=None, columns=None, target=None, namespace='namespace'):
    if columns is None:
        columns = df.columns.tolist()
    columns.remove(target)

    with open(filepath, 'w') as f:
        for _, row in tqdm(df.iterrows()):
            if namespace:
                f.write('{0} |{1} '.format(row[target], namespace))
            else:
                f.write('{0} | '.format(row[target]))
            last_feature = row.index.values[-1]
            for idx, val in row.iteritems():
                if idx not in columns:
                    continue
                if isinstance(val, str):
                    f.write('{0}'.format(val.replace(' ', '_').replace(':', '_')))
                elif (isinstance(val, float) or isinstance(val, int)):
                    if not math.isnan(val):
                        f.write('{0}:{1}'.format(idx.replace(' ', '_').replace(':', '_'), val))
                    else:
                        continue
                else:
                    f.write('{0}'.format(val.replace(' ', '_').replace(':', '_')))
                if idx != last_feature:
                    f.write(' ')
            f.write('\n')
            
def df_to_vw_classification(
    df,
    filepath='mc.txt',
    columns=None,
    target=None,
    namespace='n'
):
    if columns is None:
        columns = df.columns.tolist()
    columns.remove(target)

    with open(filepath, 'w') as f:
        for _, row in tqdm(df.iterrows()):
            if namespace:
                f.write(f"{row[target]}  |{namespace} ")
            else:
                f.write(f"{row[target]} | ")
            last_feature = columns[-1]
            for idx, val in row.iteritems():
                if idx not in columns:
                    continue
                if isinstance(val, str):
                    f.write(f"{val.replace(' ', '_').replace(':', '_')}")
                elif (isinstance(val, float) or isinstance(val, int)):
                    if not math.isnan(val):
                        f.write(f"{idx.replace(' ', '_').replace(':', '_')}:{val}")
                    else:
                        continue
                else:
                    f.write(f"{val.replace(' ', '_').replace(':', '_')}")
                if idx != last_feature:
                    f.write(' ')
            f.write('\n')
            
#df_to_vw_regression(df, filepath='data.txt', target='target')
#df_to_vw_classification(df, filepath='data.txt', target='target')

In [46]:
from sklearn.datasets import make_blobs ,make_classification
from sklearn.model_selection import train_test_split

In [47]:
features, labels = make_classification(n_samples=100000)


In [48]:
import pandas as pd

df = pd.concat([pd.DataFrame(features).add_prefix('feature_'),pd.Series(labels,name='target')],axis=1)


In [50]:
train_x,test_x,train_y,test_y = train_test_split(df.drop('target',axis=1),df['target'],)

In [53]:
len(train_x)

75000

In [54]:
df_to_vw_classification(pd.concat([train_x,train_y],axis=1), filepath='data.txt', target='target',namespace=None)

75000it [00:14, 5294.37it/s]


In [55]:
# train the dataset
!vw data.txt -f r_temp


/bin/sh: 1: vw: not found


In [56]:
df_to_vw_classification(pd.concat([test_x,test_y],axis=1), filepath='test.txt', target='target',namespace=None)

25000it [00:05, 4514.20it/s]


In [57]:
#predict on test case
! vw -t -i r_temp -p p_out test.txt

/bin/sh: 1: vw: not found


In [59]:
import numpy as np
np.genfromtxt('./p_out')

array([1.      , 0.437614, 0.337547, ..., 0.47742 , 0.      , 1.      ])

In [62]:
predicted = np.round(np.genfromtxt('./p_out')).astype('int')

In [64]:
from sklearn.metrics import accuracy_score,classification_report

In [66]:
print(classification_report(test_y,predicted))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85     12560
           1       0.83      0.89      0.86     12440

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000



In [67]:
from sklearn.ensemble import RandomForestClassifier

In [71]:
rf = RandomForestClassifier(max_depth=5)
rf.fit(train_x,train_y)
rf_predict = rf.predict(test_x)

In [72]:
print(classification_report(test_y,rf_predict))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88     12560
           1       0.86      0.91      0.89     12440

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000

